In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import utils.hash_model as image_hash_model
import pickle

# 1. 加载CIFAR-10数据集
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)


hash_bits = 64 # 假设哈希编码的位数
model_name = "vgg11"
device = torch.device("mps")
model = image_hash_model.HASH_Net(model_name, hash_bits)
model.to(device)


# 3. 加载哈希编码
with open('../classcodes/64_cifar10_10_class.pkl', 'rb') as f:
    hash_codes = torch.load(f)



criterion = QuantizationLoss()

# 5. 训练网络
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # 循环遍历数据集多次
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')
